In [1]:
import cv2

In [2]:
import tensorflow as tf

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import os, random
from tqdm import tqdm
from PIL import Image
from glob import glob

In [4]:
current_path = os.getcwd()
current_path

'/home/ichanho/workspace/dogs-vs-cats'

In [5]:
cat_list = glob(current_path + "/train/cat**", recursive=True)
dog_list = glob(current_path + "/train/dog**", recursive=True)
print(len(cat_list), len(dog_list))

12500 12500


In [6]:
print(cat_list[0], cat_list[-1])

/home/ichanho/workspace/dogs-vs-cats/train/cat.8971.jpg /home/ichanho/workspace/dogs-vs-cats/train/cat.10606.jpg


In [7]:
print(dog_list[0], dog_list[-1])

/home/ichanho/workspace/dogs-vs-cats/train/dog.365.jpg /home/ichanho/workspace/dogs-vs-cats/train/dog.11395.jpg


In [8]:
IMAGE_WIDTH = 200
IMAGE_HEIGHT = 200
IMAGE_CHANNEL = 3

In [9]:
x_train = []
x_valid = []

In [10]:
print("processing the cat image (resizing the image)")
for i in tqdm(cat_list):
    img = Image.open(i)
    img = img.convert("RGB")
    resized = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
    img_numpy = np.array(resized)
    x_train.append(img_numpy)
    x_valid.append(0)
    
print("processing the dog image (resizing the image)")
for i in tqdm(dog_list):
    img = Image.open(i)
    img = img.convert("RGB")
    resized = img.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
    img_numpy = np.array(resized)
    x_train.append(img_numpy)
    x_valid.append(1)

processing the cat image (resizing the image)


100%|██████████| 12500/12500 [01:21<00:00, 152.99it/s]


processing the dog image (resizing the image)


100%|██████████| 12500/12500 [01:24<00:00, 147.66it/s]


In [11]:
X = np.array(x_train)
Y = pd.get_dummies(x_valid)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

(25000, 200, 200, 3)
(25000, 2)


In [12]:
model = tf.keras.Sequential()

In [13]:
# 1st
model.add(Conv2D(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNEL), kernel_size=(3, 3), filters=32, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
# 2nd
model.add(Conv2D(kernel_size=(3,3),filters=64, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
# 3rd
model.add(Conv2D(kernel_size=(3,3),filters=128, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

In [16]:
# Fully Connected
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))
model.add(Dense(2, activation="softmax"))

In [17]:
model.compile('Adam', 'categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 198, 198, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 97, 97, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 128)       7

In [ ]:
model.fit(X, Y, epochs=25, validation_split=0.2)

Epoch 1/25
625/625 [==============================] - 87s 139ms/step - loss: 0.0141 - accuracy: 0.9947 - val_loss: 1.3572 - val_accuracy: 0.7346
Epoch 2/25
625/625 [==============================] - 83s 133ms/step - loss: 0.0228 - accuracy: 0.9929 - val_loss: 0.8344 - val_accuracy: 0.8236
Epoch 3/25
625/625 [==============================] - 83s 133ms/step - loss: 0.0167 - accuracy: 0.9939 - val_loss: 1.2176 - val_accuracy: 0.7580
Epoch 4/25
625/625 [==============================] - 83s 132ms/step - loss: 0.0154 - accuracy: 0.9952 - val_loss: 1.0036 - val_accuracy: 0.7892
Epoch 5/25
625/625 [==============================] - 83s 132ms/step - loss: 0.0192 - accuracy: 0.9938 - val_loss: 0.5635 - val_accuracy: 0.8522
Epoch 6/25
625/625 [==============================] - 83s 132ms/step - loss: 0.0186 - accuracy: 0.9935 - val_loss: 2.0862 - val_accuracy: 0.6202
Epoch 7/25
625/625 [==============================] - 83s 132ms/step - loss: 0.0167 - accuracy: 0.9937 - val_loss: 1.6118 - val_ac